## Importing Libraries

 <!-- *************************Importing Libraries************************* -->

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt #for visualisation
import seaborn as sns

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Image Preprocessing

Training image preprocessing

In [2]:
#from keras documnetation
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels = "inferred", #directory name are considered labels
    label_mode = "categorical", #int is default, instead use categorial vector, as we are not solving 2 class problem, we have multiple classes
    class_names = None, #as already we are using labels from the directory
    color_mode = 'rgb',
    batch_size = 32, #if want to speed up , take 64 or more, at a time we are feeding 
    image_size = (128, 128), 
    shuffle = True, #at the time of feeding to the model, we will shuffle the dataset so as to reduce biased-ness
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = "bilinear", 
    follow_links = False,
    crop_to_aspect_ratio = False,
)

Found 43444 files belonging to 38 classes.


Validation image preprocessing

In [3]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels = "inferred",
    label_mode = "categorical",
    class_names = None,
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (128, 128), 
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = "bilinear", 
    follow_links = False,
    crop_to_aspect_ratio = False,
)

Found 10861 files belonging to 38 classes.


In [4]:
print(training_set) #summary of the training set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>


In [5]:
for x,y in training_set: 
    print(x, x.shape)
    print(y,y.shape)
    break #seeing for just 1st batch

tf.Tensor(
[[[[158.75 153.75 149.75]
   [156.25 151.25 147.25]
   [167.25 162.25 158.25]
   ...
   [194.75 192.75 193.75]
   [203.   201.   202.  ]
   [201.5  199.5  200.5 ]]

  [[153.5  148.5  144.5 ]
   [158.5  153.5  149.5 ]
   [168.75 163.75 159.75]
   ...
   [197.75 195.75 196.75]
   [203.25 201.25 202.25]
   [203.25 201.25 202.25]]

  [[158.5  153.5  149.5 ]
   [163.   158.   154.  ]
   [164.   159.   155.  ]
   ...
   [199.5  197.5  198.5 ]
   [202.5  200.5  201.5 ]
   [215.25 213.25 214.25]]

  ...

  [[131.25 123.25 121.25]
   [129.5  121.5  119.5 ]
   [151.   143.   141.  ]
   ...
   [179.5  175.5  176.5 ]
   [162.5  158.5  159.5 ]
   [176.25 172.25 173.25]]

  [[148.5  140.5  138.5 ]
   [145.75 137.75 135.75]
   [133.25 125.25 123.25]
   ...
   [168.5  164.5  165.5 ]
   [181.5  177.5  178.5 ]
   [176.25 172.25 173.25]]

  [[139.75 131.75 129.75]
   [147.5  139.5  137.5 ]
   [129.5  121.5  119.5 ]
   ...
   [161.5  157.5  158.5 ]
   [163.5  159.5  160.5 ]
   [173.75 169.75 17

In [6]:
print(validation_set) #summary of the validation set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>


## Building Model

In [7]:
from tensorflow.keras.layers import Dense,Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [8]:
model = Sequential()

## Building convolutional layer and pooling layer

In [9]:
model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = [128, 128, 3]))
model.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))

In [10]:
model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))

In [11]:
model.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))

In [12]:
model.add(Conv2D(filters = 256, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 256, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))

In [13]:
model.add(Conv2D(filters = 512, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 512, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(pool_size = 2, strides = 2))

In [14]:
model.add(Dropout(0.25)) #to avoid overfitting, dropping 24% of the neurons

## Flatten the output

In [15]:
model.add(Flatten())

In [16]:
model.add(Dense(units = 1500, activation = 'relu')) #changed from 1024 to 1500

In [17]:
#output layer
model.add(Dense(units = 38, activation = 'softmax'))

In [18]:
model.add(Dropout(0.4)) #dropping 40%of the neurons

## Compiling model

In [34]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])#decreased learning rate frmo 0.001 to 0.0001 

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                   

## Model Training

In [36]:
training_history = model.fit(x = training_set, validation_data = validation_set, epochs = 10)

Epoch 1/10
   9/1358 [..............................] - ETA: 11:46 - loss: nan - accuracy: 0.0208

KeyboardInterrupt: 

## Model evaluation

In [ ]:
#model evaluation on training set
training_loss, training_accuracy = model.evaluate(training_set)


In [ ]:
print(training_loss, training_accuracy)

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_set)

In [ ]:
print(validation_loss, validation_accuracy)

## Saving Model

In [ ]:
model.save("trained_model.h5")

## Visualising the history

In [ ]:
print(training_history.history)

## Saving the history in json

In [ ]:
import json
with open("training_history.json", "w") as f:
    json.dump(training_history.history, f)